# Оптимизированный пайплайн для двух задач: предсказание уровня удовлетворенности и увольнений сотрудников

In [ ]:

# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression
import shap
import matplotlib.pyplot as plt

# Определим функцию для вычисления SMAPE (симметричное среднее абсолютное процентное отклонение)
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator != 0)
    return np.mean(diff) * 100

# Создаем пользовательскую метрику SMAPE
smape_scorer = make_scorer(smape, greater_is_better=False)

# Загрузка данных
job_X_train = pd.read_csv('datasets/train_job_satisfaction_rate.csv')
job_X_test = pd.read_csv('datasets/test_features.csv')
job_y_test = pd.read_csv('datasets/test_target_job_satisfaction_rate.csv')
quit_X_train = pd.read_csv('datasets/train_quit.csv')
quit_y_test = pd.read_csv('datasets/test_target_quit.csv')

# Подготовка данных для задачи №1 (предсказание уровня удовлетворённости)
job_X_train_ml = job_X_train.copy().set_index('id')
job_y_train_ml = job_X_train_ml.pop('job_satisfaction_rate')

# Подготовка данных для задачи №2 (предсказание увольнений)
quit_X_train_ml = quit_X_train.copy().set_index('id')
quit_y_train_ml = quit_X_train_ml.pop('quit')

# Заполним пропуски и исправим опечатки
job_X_train_ml['level'].replace('sinior', 'senior', inplace=True)
quit_X_train_ml['level'].replace('sinior', 'senior', inplace=True)


### Пайплайн для задачи №1 (Предсказание уровня удовлетворённости сотрудников)

In [ ]:

# Категориальные, порядковые и числовые признаки
categorical_features = ['dept', 'last_year_promo', 'last_year_violations']
ordinal_features = ['level', 'workload']
numeric_features = ['employment_years', 'supervisor_evaluation', 'salary']

# Пайплайн для числовых признаков с масштабированием
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Пайплайн для категориальных признаков с OHE-кодированием
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))])

# Пайплайн для порядковых признаков с OrdinalEncoder
ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder())])

# Объединяем все шаги предобработки в один ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)])

# Модель для регрессии - LGBM
regressor = lgb.LGBMRegressor(random_state=42)

# Пайплайн для задачи 1
pipeline_1 = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', regressor)])

# Гиперпараметры для поиска
param_grid_1 = {
    'regressor__max_depth': range(1, 50),
    'regressor__n_estimators': range(10, 100),
    'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler()]
}

# RandomizedSearchCV для поиска оптимальных гиперпараметров для задачи 1
search_1 = RandomizedSearchCV(pipeline_1, param_grid_1, n_iter=50, scoring=smape_scorer, cv=5, random_state=42)
search_1.fit(job_X_train_ml, job_y_train_ml)

# Лучшие параметры и метрика SMAPE на кросс-валидации
best_model_1 = search_1.best_estimator_
best_smape = search_1.best_score_
best_smape


### Пайплайн для задачи №2 (Предсказание увольнений сотрудников)

In [ ]:

# Для задачи №2 (увольнения) добавим предсказанный признак job_satisfaction_rate
quit_train_J = best_model_1.predict(quit_X_train_ml)
quit_X_train_ml['job_satisfaction_rate'] = quit_train_J

# Подготовка пайплайна для задачи 2
# Добавляем 'job_satisfaction_rate' в список числовых признаков
numeric_features_2 = numeric_features + ['job_satisfaction_rate']

# Пайплайн для задачи 2
pipeline_2 = Pipeline(steps=[('preprocessor', preprocessor),
                             ('classifier', LogisticRegression(solver='liblinear', penalty='l1', random_state=42))])

# Гиперпараметры для поиска
param_grid_2 = {
    'classifier__C': [0.1, 1, 10, 100],
    'preprocessor__num__scaler': [StandardScaler(), MinMaxScaler()]
}

# RandomizedSearchCV для поиска гиперпараметров задачи 2
search_2 = RandomizedSearchCV(pipeline_2, param_grid_2, n_iter=50, scoring='roc_auc', cv=5, random_state=42)
search_2.fit(quit_X_train_ml, quit_y_train_ml)


### Важность признаков с помощью SHAP

In [ ]:

# Анализ важности признаков с использованием SHAP
model = search_2.best_estimator_.named_steps['classifier']
X_test_transformed = search_2.best_estimator_.named_steps['preprocessor'].transform(quit_X_train_ml)
explainer = shap.LinearExplainer(model, X_test_transformed)
shap_values = explainer.shap_values(X_test_transformed)
shap.summary_plot(shap_values, X_test_transformed, feature_names=numeric_features_2)
